### **Import of Libraries**

In [1]:
# Import of all relevant libraries
import pandas as pd 
import numpy as np
from skimpy import clean_columns
import re

### **Data Import & Exploration**

In [2]:
# Import of the .csv-file with latin1 encoding, since the utf-8 did not work
import_file = '/Users/benediktgeitz/Desktop/SIS_Faculty-List.csv'
df = pd.read_csv(import_file, encoding= 'latin1')

In [ ]:
# Give insight in the count, uniqueness and top frequency of values of each column
df.describe()

In [ ]:
# List information of each column, including non-null count and data type
df.info()

In [ ]:
# Get an overview of the upper ten records of each column
df.head(10)

In [ ]:
# List every column with its sum of missing values
df.isnull().sum()

In [ ]:
# List every column with its percentage of missing values
(df.isnull().sum() / len(df)) * 100

In [ ]:
# Get all duplicated records with respect to the column 'Name'
df[df.duplicated(['Name'], keep=False)]

In [350]:
# transform columns into upper cases only -done
# Rename columns with / or - Space ? (e.g. Join\nDate ;Reports To; Highest/nQualification/nLevel; Highest Qualification; All Qualifications from Profile; Courses Taught- Term 201510; DOCUMENT OTHER PROFESSIONAL CERTIFICATION CRITIERA) -done
# Filter ID's with 0 instead of LT... (40 ID's with 0) -done (created random, unique new LT-ID's)
# Other date format --> at least with yyyy format -done
# Are there any columns completely empty/NaN/only with one unique record? (transformed LWD into HAS_LEFT with category-type values 1 & 0 --> 0 =still there, 1 =has left)
# NaN filtering ('REPORTS_TO' value is directly linked to the 'LOCATION' column --> used to impute missing data; and the other way round / cleaned JOIN_DATE with placeholder "1900-01-01")
# 'ALL_QUALIFICATIONS_FROM_PROFILE' mit 'HIGHEST_QUALIFICATION' füllen -done
# wenn 2 Werte und mehr fehlen, Eintrag kicken -done
# 'MAJOR_TEACHING_FIELD' mit 'COURSES_TAUGHT_TERM_201510' -not possible
# 'HIGHEST_QUALIFICATION_LEVEL' auf Abschluss bereinigen
# Namen in Vorname und Nachname trennen? -done
# Mit 'HIGHEST_QUALIFICATION_LEVEL' und 'HIGHEST_QUALIFICATION' 'MAJOR' füllen? -done
# Handle records with more than one entry devided by slash or paragraph or parantheses -done
# "Industry & Teaching experience" records ok because of consistency or still bad because of "&"? -done
# Some courses have codes, some not - some have a different order -done
# Duplicates? (Zikida Koudou; Amanda Hobson exist twice, each with a different academic degree per record) -done
# Drop 'TYPE', 'DIVISION' --> one value for all records -done

### **Data Cleaning**

In [225]:
# Make all column names upper case and extinguish -/&() etc.
clean_df = clean_columns(df, case = 'const')

In [ ]:
# Transform JOIN_DATE into a datetime format
clean_df['JOIN_DATE'] = pd.to_datetime(clean_df['JOIN_DATE'], format='%d-%b-%y')

# Replace NaN join date with clearly out of range date 
clean_df['JOIN_DATE'].fillna(pd.Timestamp('1900-01-01'), inplace=True)

In [229]:
# Function to generate a random and new ID for missing IDs
def generate_id(existing_ids, prefix='LT', length=9):
    while True:
        new_id = prefix + ''.join(np.random.choice(list('0123456789'), size=length-2))
        if new_id not in existing_ids:
            return new_id

# Get the list of existing IDs
existing_ids = clean_df['ID'][clean_df['ID'] != '0'].dropna().tolist()

# Fill '0' and NaN IDs with new generated IDs
clean_df['ID'] = clean_df['ID'].apply(lambda x: generate_id(existing_ids) if x == '0' or pd.isna(x) else x)

In [230]:
# The column LWD (Last Working Day) has too few records, so it wisely to generate a binary code with 1 for has left and 0 for is still working.

# Convert 'LWD' to datetime
clean_df['LWD'] = pd.to_datetime(clean_df['LWD'], errors='coerce')

# Create the binary feature indicating if the employee has already left
clean_df['HAS_LEFT'] = clean_df['LWD'].notna().astype(int)

# Convert 'HAS_LEFT' to categorical type for future aggregation views
clean_df['HAS_LEFT'] = clean_df['HAS_LEFT'].astype('category')

# Drop the original 'LWD' column since it's no longer needed
clean_df.drop(columns=['LWD'], inplace=True)

/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/312193449.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clean_df['LWD'] = pd.to_datetime(clean_df['LWD'], errors='coerce')


In [231]:
# Drop the 'DIVISION' and 'TYPE' columns since all records are Full-Time contracts and from business division
clean_df.drop(columns=['TYPE', 'DIVISON'], inplace=True)

In [233]:
# Define a dictionary for replacements
replacement_dict = {
    'Faculty - Business': 'Business',
    'Faculty - Business (Admin)': 'Business Admin',
    'Adjunct Faculty - Business': 'Adjunct Business',
    'Faculty-Business (HR & Admin.)': 'Business HR Admin',
    'Faculty - Business ': 'Business',
    'Faculty - Trainee': 'Trainee',
    'Faculty - Business & Comp Lit Faculty': 'Business & Comp Lit',
    'Faculty- Business': 'Business',
    'Faculty-Business': 'Business',
    'Business, Comp & Math Faculty': 'Business, Computer & Maths',
    'Graduate Trainee': 'Graduate Trainee',
    'Faculty - Business & Comp Lit.': 'Business & Comp Lit'
}

# Function to clean title names using the replacement dictionary
def clean_title(title):
    # Remove leading/trailing spaces
    title = title.strip()
    
    # Remove the term 'Faculty'
    title = re.sub(r'\bFaculty\b', '', title, flags=re.IGNORECASE)
    
    # Remove extra spaces left after removing 'Faculty'
    title = re.sub(r'\s+', ' ', title)
    title = title.strip('- ')
    
    # Standardize common variations
    title = re.sub(r'\s*-\s*', '-', title)  # Remove spaces around hyphens
    title = re.sub(r'\s*&\s*', ' & ', title)  # Add spaces around '&'
    title = re.sub(r'\s+', ' ', title)  # Replace multiple spaces with a single space
    
    # Replace using the dictionary
    return replacement_dict.get(title, title)

# Apply the cleaning function to the TITLE column
clean_df['TITLE'] = clean_df['TITLE'].apply(clean_title)

In [234]:
# Unique values of the 'LOCATION' columnn
clean_df['LOCATION'].unique()

array(['London', 'Liverpool', 'Manchester', nan, 'Bristol', 'Brighton',
       'Cardiff', 'Cardiff ', 'Belfast', 'Glasgow', 'Leeds', 'Bolton',
       'Birmingham', 'Oxford', 'Southampton', 'Leicester', 'Cambridge'],
      dtype=object)

In [235]:
# Visualisation of every record of the 'LOCATON' column that is equal to 'Southampton'
clean_df.loc[clean_df['LOCATION'] == 'Southampton']

,ID,NAME,LOCATION,GRADE,TITLE,JOIN_DATE,REPORTS_TO,HIGHEST_QUALIFICATION_LEVEL,HIGHEST_QUALIFICATION,MAJOR,UNIVERSITY,ALL_QUALIFICATIONS_FROM_PROFILE,COURSES_TAUGHT_TERM_201510,MAJOR_TEACHING_FIELD,DOCUMENT_OTHER_PROFESSIONAL_CERTIFICATION_CRITIERA_FIVE_YEARS_WORK_EXPERIENCE_TEACHING_EXCELLENCE_PROFESSIONAL_CERTIFICATIONS,CRITERIA,HAS_LEFT
231,LT9001206,Twanda Chumbley,Southampton,FA,Business,1997-08-09,Ian Westwood,Masters,Master of Business Administrat,Business Administration,The Waikato University,Master of Business Administrat (Business Admin...,BUS3103 - Internat Busi Globalisation\nBUS3903...,Accounting and General Business,20 years government and industry experience in...,Industry & Teaching experience,0
232,LT9305072,Klaus Pontius,Southampton,FA,Business,2014-10-19,Ian Westwood,Masters,Master of Business Administrat,Business Administration,Open University,"Bachelor (Geology), Master of Business Adminis...",BUS4583 - ISO Standards and Excellence\nBUS455...,OB/HR/ MKT/General Management,10 years of Industry Experience and 14 years o...,Industry & Teaching experience,0
233,LT9300994,Roselyn Paulos,Southampton,FA,Business,2014-08-17,Ian Westwood,Masters,Master of Business Administrat,Business Administration,Yarmouk University,Master of Business Administrat (Business Admin...,BUS4956 - Integrative Industry Project\nBUS340...,Business Administration/Strategic Management/o...,8 years industrial experience;5 years in produ...,Industry & Teaching experience,0


In [236]:
# Create a mapping of locations to the most common supervisor in each location
location_supervisors = clean_df.groupby('LOCATION')['REPORTS_TO'].apply(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').to_dict()

# Impute missing values based on the location
clean_df['REPORTS_TO'] = clean_df.apply(lambda row: location_supervisors[row['LOCATION']] if pd.isnull(row['REPORTS_TO']) else row['REPORTS_TO'], axis=1)

# Create a mapping of supervisor to the most common location in each supervisor
supervisors_location = clean_df.groupby('REPORTS_TO')['LOCATION'].apply(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').to_dict()

# Impute missing values based on the location
clean_df['LOCATION'] = clean_df.apply(lambda row: supervisors_location[row['REPORTS_TO']] if pd.isnull(row['LOCATION']) else row['LOCATION'], axis=1)


In [237]:
# Function to split the name into forename and surname
def split_name(name):
    parts = name.split()
    if len(parts) == 1:
        forename = parts[0]
        surname = ''
    else:
        forename = ' '.join(parts[:-1])
        surname = parts[-1]
    return pd.Series([forename, surname])

# Apply the function to split the NAME column
clean_df[['FORENAME', 'SURNAME']] = clean_df['NAME'].apply(split_name)

# Drop the original NAME column if needed
clean_df.drop(columns=['NAME'], inplace=True)

# Reorder columns to place FORENAME and SURNAME after ID
cols = clean_df.columns.tolist()
cols.insert(1, cols.pop(cols.index('FORENAME')))
cols.insert(2, cols.pop(cols.index('SURNAME')))
clean_df = clean_df[cols]

In [238]:
# Function to extract qualifications
def extract_qualifications(text):
    bachelor_details = 'no Bachelor'
    master_details = 'no Master'
    diploma_details = 'no Diploma'
    certificate_details = 'no Certificate'
    phd_details = 'no Ph.D'
    
    if isinstance(text, str):
        entries = text.split(',')
        for entry in entries:
            entry = entry.strip()
            if 'Bachelor' in entry:
                bachelor_details = entry
            elif 'Master' in entry or 'Masters' in entry:
                master_details = entry
            elif 'Diploma' in entry:
                diploma_details = entry
            elif 'Certificate' in entry:
                certificate_details = entry
            elif 'Ph.D' in entry or 'Doctorate' in entry:
                phd_details = entry
    
    return pd.Series([bachelor_details, master_details, diploma_details, certificate_details, phd_details])

# Apply the function to the ALL_QUALIFICATIONS_FROM_PROFILE column
clean_df[['Bachelor_Details', 'Master_Details', 'Diploma_Details', 'Certificate_Details', 'Ph.D_Details']] = clean_df['ALL_QUALIFICATIONS_FROM_PROFILE'].apply(extract_qualifications)

# Drop the original column if needed
clean_df.drop(columns=['ALL_QUALIFICATIONS_FROM_PROFILE'], inplace=True)


/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/2784429788.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[['Bachelor_Details', 'Master_Details', 'Diploma_Details', 'Certificate_Details', 'Ph.D_Details']] = clean_df['ALL_QUALIFICATIONS_FROM_PROFILE'].apply(extract_qualifications)
/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/2784429788.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[['Bachelor_Details', 'Master_Details', 'Diploma_Details', 'Cer

In [239]:
# Define the academic ranking
degree_rank = {
    'Certificate': 1,
    'Diploma': 2,
    'Bachelor': 3,
    'Master': 4,
    'Ph.D': 5
}

# Function to determine the highest degree
def get_highest_degree(row):
    degrees = {
        'Ph.D': row['Ph.D_Details'],
        'Master': row['Master_Details'],
        'Bachelor': row['Bachelor_Details'],
        'Diploma': row['Diploma_Details'],
        'Certificate': row['Certificate_Details']
    }
    
    # Initialize highest_degree with the lowest possible rank
    highest_degree = 'no degree'
    highest_rank = 0
    
    for degree, details in degrees.items():
        if details and details != f'no {degree}':
            if degree_rank[degree] > highest_rank:
                highest_rank = degree_rank[degree]
                highest_degree = degree
    
    return highest_degree

# Apply the function to determine the highest qualification level
clean_df['HIGHEST_QUALIFICATION_LEVEL'] = clean_df.apply(get_highest_degree, axis=1)


/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/3402771132.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['HIGHEST_QUALIFICATION_LEVEL'] = clean_df.apply(get_highest_degree, axis=1)


In [240]:
# Define the academic ranking
degree_rank = {
    'Certificate': 1,
    'Diploma': 2,
    'Bachelor': 3,
    'Master': 4,
    'Ph.D': 5
}

# Function to determine the highest degree description
def get_highest_degree_description(row):
    degrees = {
        'Ph.D': row['Ph.D_Details'],
        'Master': row['Master_Details'],
        'Bachelor': row['Bachelor_Details'],
        'Diploma': row['Diploma_Details'],
        'Certificate': row['Certificate_Details']
    }
    
    # Initialize highest_description with the lowest possible rank
    highest_description = ''
    highest_rank = 0
    
    for degree, details in degrees.items():
        if details and details != f'no {degree}':
            if degree_rank[degree] > highest_rank:
                highest_rank = degree_rank[degree]
                highest_description = details
    
    return highest_description

# Apply the function to determine the highest qualification description
clean_df['HIGHEST_QUALIFICATION'] = clean_df.apply(get_highest_degree_description, axis=1)


/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/3744534144.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['HIGHEST_QUALIFICATION'] = clean_df.apply(get_highest_degree_description, axis=1)


In [241]:
# Unique values of the 'COURSES_TAUGHT_TERM_201510' column
clean_df['COURSES_TAUGHT_TERM_201510'].unique()

array(['BUS2403 - Innovation & Entrepreneurship\nBUS2003 - Business and Commercial Law\nLSG2013 - Globalization, Media & Soc\nBUS1003 - Management and Leadership',
       'BUS4173 - Managerial Accounting II\nBUS4916 - Integrative Industry Project\nBUS4163 - Taxation\nBUS4133 - Managerial Accounting I',
       'BUS3903 - Contemporary Business\nBUS1303 - Marketing',
       'HRM, Organizational behaviour, Strategic Mgt, project mgt, Risk mgt',
       'Micro and Macroeconomics, Statistics,Introduction to Business',
       'BUS3103 - Internat Busi Globalisation\nBUS1303 - Marketing\nBUS4543 - Quality Management Tools\nBUS2103 - Operations Management',
       'BUS2403 - Innovation & Entrepreneurship\nBUS2303 - Financial Management\nBUS1303 - Marketing\nBUS2103 - Operations Management',
       'BUS4956 - Integrative Industry Project\nBUS3903 - Contemporary Business\nBUS1403 - Business Ethics and Corporate\nBUS4543 - Quality Management Tools',
       'BUS3103 - Internat Busi Globalisation\nBUS

In [242]:
# Drop rows with 2 or more missing values
# thresh = number of columns - 1 means at least (number of columns - 1) non-NA values to keep the row
final_clean_df = clean_df.dropna(thresh=clean_df.shape[1] - 1)

In [250]:
# Function to extract years of experience and certifications
def extract_experience_and_certifications(text):
    teaching_exp = 0
    professional_exp = 0
    certifications = []
    
    if isinstance(text, str):
        # Normalize multiple spaces to a single space for consistent splitting
        text = re.sub(r'\s+', ' ', text)
        
        # Extract teaching experience
        teaching_matches = re.findall(r'(\d+)\+?\s*years?\s*(?:of\s*)?teaching', text, re.IGNORECASE)
        teaching_exp = sum(int(match) for match in teaching_matches)
        
        # Extract professional experience
        professional_matches = re.findall(r'(\d+)\+?\s*years?\s*(?:of\s*)?(?:professional|work|management|corporate)', text, re.IGNORECASE)
        professional_exp = sum(int(match) for match in professional_matches)
        
        # Extract certifications
        certifications_matches = re.findall(r'(?:certified|certification|cert\.|certificates?|certified)\s*[:\s]*([A-Za-z0-9\s\-\(\)]+)', text, re.IGNORECASE)
        for match in certifications_matches:
            certifications.append(match.strip())
        
        # Special cases: handle variations in wording
        if 'PMP' in text:
            certifications.append('PMP')
        if 'FDP' in text:
            certifications.append('FDP')
        if 'ICDL' in text:
            certifications.append('ICDL')
        if 'TAFE Certificate IV' in text:
            certifications.append('TAFE Certificate IV')
    
    return pd.Series([teaching_exp, professional_exp, ', '.join(certifications)])

# Apply the function to the DataFrame
final_clean_df[['TEACHING_EXPERIENCE_YEARS', 'PROFESSIONAL_EXPERIENCE_YEARS', 'CERTIFICATIONS']] = final_clean_df['DOCUMENT_OTHER_PROFESSIONAL_CERTIFICATION_CRITIERA_FIVE_YEARS_WORK_EXPERIENCE_TEACHING_EXCELLENCE_PROFESSIONAL_CERTIFICATIONS'].apply(extract_experience_and_certifications)

# Drop the original column
final_clean_df.drop(columns=['DOCUMENT_OTHER_PROFESSIONAL_CERTIFICATION_CRITIERA_FIVE_YEARS_WORK_EXPERIENCE_TEACHING_EXCELLENCE_PROFESSIONAL_CERTIFICATIONS'], inplace=True)

# Display the cleaned DataFrame

/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/1697925232.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_clean_df[['TEACHING_EXPERIENCE_YEARS', 'PROFESSIONAL_EXPERIENCE_YEARS', 'CERTIFICATIONS']] = final_clean_df['DOCUMENT_OTHER_PROFESSIONAL_CERTIFICATION_CRITIERA_FIVE_YEARS_WORK_EXPERIENCE_TEACHING_EXCELLENCE_PROFESSIONAL_CERTIFICATIONS'].apply(extract_experience_and_certifications)
/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/1697925232.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [251]:
# List of valid certifications
valid_certifications = [
    'GCMA', 'Trainer', 'ICDL',
    'Hospitality Educator(American Hotleirs and Lodging Association)',
    'FDP', 'PMP',
    'IV in Training and Assessment', 'TAFE Certificate IV',
    'Member of the Institute of Leadership and Management',
    'IV in Training and Assessment (Australia)',
    'Certified Islamic Finance Expert', 'Takaful Professional',
    'Certified Attorney', 'Notary Public', 'Certified Conveyancer',
    'Professional Trainer (CAP)', 'ISW',
    'Certified Supply Chain Manager',
    'ISO 9000 Lead Assessor', 'Associate of ICSA (Singapore-UK)',
    'Certificate of Entrepenuership from Cambridge University',
    'CIM', 'Webmaster',
    'Certified Hotel Administration American Hotel Motel Association Educational Institute',
    'IV Workplace Training and Assessment',
    'ICDL', 'Professional Trainer (CTPP)', 'IV Workplace Training - Australia', 'Trainer - Phi Theta Kapa International Honor Society - USA',
    'Professional(SCP)',
    'IV in Training', 'Practitioner with Prince 2 Introductory PMBOK (Project Management Body of Knowledge) Six Sigma',
    'Registered Cost Accountant(UK) and Accounting Technician(UK)',
    'Certified Manangement Account- CMA',
    'Financial Planner (CFP)', 'Management Accountant (CMA)',
    'III for Professional Workplace Coaches', 'Train the Trainer'
]

# Function to clean certifications
def clean_certifications(certifications):
    if pd.isna(certifications):
        return ''
    
    cleaned_certs = []
    for cert in valid_certifications:
        if cert in certifications:
            cleaned_certs.append(cert)
    
    return ', '.join(cleaned_certs)

# Apply the function to the CERTIFICATIONS column
final_clean_df['CERTIFICATIONS'] = final_clean_df['CERTIFICATIONS'].apply(clean_certifications)

# Replace empty strings with "no certificate"
final_clean_df['CERTIFICATIONS'] = final_clean_df['CERTIFICATIONS'].replace('', 'no certificate')

/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/3691390259.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_clean_df['CERTIFICATIONS'] = final_clean_df['CERTIFICATIONS'].apply(clean_certifications)


In [253]:
# Several information about the data set, including data types, non-null values and column names
final_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222 entries, 0 to 283
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   ID                             222 non-null    object        
 1   FORENAME                       222 non-null    object        
 2   SURNAME                        222 non-null    object        
 3   LOCATION                       222 non-null    object        
 4   GRADE                          222 non-null    object        
 5   TITLE                          222 non-null    object        
 6   JOIN_DATE                      222 non-null    datetime64[ns]
 7   REPORTS_TO                     222 non-null    object        
 8   HIGHEST_QUALIFICATION_LEVEL    222 non-null    object        
 9   HIGHEST_QUALIFICATION          222 non-null    object        
 10  MAJOR                          219 non-null    object        
 11  UNIVERSITY              

In [ ]:
# Percentage of missing data per column
(final_clean_df.isnull().sum() / len(final_clean_df)) * 100

In [258]:
# Function to split and standardize course names
def split_courses(courses):
    if pd.isna(courses):
        return []
    # Split by new lines, commas, and other delimiters
    return re.split(r'[\n,]+', courses.strip())

# Apply the function to the COURSES_TAUGHT_TERM_201510 column
final_clean_df['COURSES_SPLIT'] = final_clean_df['COURSES_TAUGHT_TERM_201510'].apply(split_courses)

# Flatten the list of all courses to get unique courses
all_courses = final_clean_df['COURSES_SPLIT'].explode().dropna().unique()

# Remove leading and trailing spaces from course names
all_courses = [course.strip() for course in all_courses]

# Create binary columns for each course
for course in all_courses:
    final_clean_df[course] = final_clean_df['COURSES_SPLIT'].apply(lambda x: int(course in x)).astype('category')

# Drop the original and intermediate columns if needed
final_clean_df.drop(columns=['COURSES_TAUGHT_TERM_201510', 'COURSES_SPLIT'], inplace=True)

/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/956627779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_clean_df['COURSES_SPLIT'] = final_clean_df['COURSES_TAUGHT_TERM_201510'].apply(split_courses)
/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/T/ipykernel_61679/956627779.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_clean_df[course] = final_clean_df['COURSES_SPLIT'].apply(lambda x: int(course in x)).astype('category')
/var/folders/0s/q8l2rp352n5_z75xdr9kw1780000gn/

In [260]:
# Export of the final data set in Excel format 
final_clean_df.to_excel('final_dataset.xlsx')